In [ ]:
# usual imports
import os
import numpy as np
import matplotlib.pyplot as plt

from rail.core.utils import RAILDIR
#from rail.estimation.algos.bpz_version.utils import RAIL_BPZ_DIR
from rail.pipelines.estimation.estimate_all import EstimatePipeline
from rail.core import common_params

from rail.pipelines.utils.name_factory import NameFactory, DataType, CatalogType, ModelType, PdfType
#from rail.core.stage import RailStage, RailPipeline

import qp

#import ceci

In [ ]:
bands = 'grizy'
maglims = [27.66, 27.25, 26.6, 26.24, 25.35]
maglim_dict={}
for band,limx in zip(bands, maglims):
    maglim_dict[f"HSC{band}_cmodel_dered"] = limx

In [ ]:
common_params.set_param_defaults(
    bands=[f'HSC{band}_cmodel_dered' for band in bands],
    err_bands=[f'{band}_cmodel_magerr' for band in bands],
    nondetect_val=np.nan,
    ref_band='HSCi_cmodel_dered',
    redshift_col='specz_redshift',
    mag_limits=maglim_dict,
    zmax=6.0,
)

In [ ]:
pipe = EstimatePipeline()

In [ ]:
namer = NameFactory()

In [ ]:
datadir = '/Users/jtm/data/lsst/PZ/'
input_file = os.path.join(datadir, 'dered_223501_sz_match_pdr3_dud_NONDET.hdf5') # hsc_train/

input_dict = dict(
    model_fzboost=os.path.join(namer.get_data_dir(DataType.model, ModelType.estimator), "model_FZBoost.hdf5"), #_fzboost                                                
    model_trainz=os.path.join(namer.get_data_dir(DataType.model, ModelType.estimator), "model_trainz.pkl"),
    input=input_file,
)

In [ ]:
pipe_info = pipe.initialize(input_dict, dict(output_dir='.', log_dir='.', resume=True), None)

In [ ]:
pipe.save('tmp_estimate_all.yml')

In [ ]:
import ceci
pr = ceci.Pipeline.read('tmp_estimate_all.yml')

In [ ]:
pr.run()

In [ ]:
infile = '/Users/jtm/repositories/rail_pipelines/nb/estimation/pdf/pz/output_FZBoost.hdf5'

qp_ensemble = qp.read(infile, )

print(qp_ensemble.npdf)
print(qp_ensemble.ancil)

In [ ]:
qp_ensemble.dist.yvals.shape

In [ ]:
qp_ensemble.rvs(size=1)

In [ ]:
qp_ensemble[4].plot_native(xlim=(0,3))

In [ ]:
qp_ensemble.pdf(0.5)

In [ ]:
plt.figure()
plt.hist(qp_ensemble.ancil['zmode'], bins=40)

In [ ]:
# from rail.core import RailStage
# dir(RailStage)
# DS = RailStage.data_store
# DS.keys()